# Parse

In [3]:
import os

# READ IN ALL .tsv FILE PATHS AND ADD THEM TO THE LIST 'tsv_file_paths'

tsv_file_paths = []
for root, subdirs, files in os.walk("/geosat2/julaiti/tsv_all/SIO"):
    tsv_file_paths += [os.path.join(root, filename)
                  for filename in files if filename.endswith("tsv")]
len(tsv_file_paths)

254

In [4]:
cols = "longitude latitude depth sigh sigd SID predicted_depth ID d10 d20 d60 seafloor_age curvature(VGG) spreading_rate sediment_thickness seafloor_roughness NDP_@2.5am NDP_@5am NDP_@10am NDP_@30am STD_@2.5am STD_@5am STD_@10am STD_@30am depth_SUB_median@2.5am depth_SUB_median@5am depth_SUB_median@10am depth_SUB_median@30am year data_type".split()

In [6]:
for filename in tsv_file_paths:
    # SET OUTPUT FILE NAME AS "INPUT".libsvm
    svm_fn = filename.rsplit(".", 1)[0] + ".libsvm"
    
    # OPEN THE TSV FILE
    with open(filename) as fread:
        # OPEN THE .libsvm FILE
        with open(svm_fn, 'w') as fwrite:
            for line in fread:
                cols = line.strip().split()
                if not cols:
                    continue
                label = (cols[4] == '9999')
                cols = cols[:4] + cols[5:]
                labels = ["%d:%s" % (i, v) for i, v in enumerate(cols) if v.lower() != "nan" and v != "0"]
                fwrite.write("%d %s\n" % (label, ' '.join(labels)))

In [ ]:
with open("./all-done.txt", 'w') as f:kk''
    
    
    ]
    f.write("done\n")

# Collect

In [ ]:
import os

libsvm_file_paths = []
for root, subdirs, files in os.walk("./"):
    libsvm_file_paths += [os.path.join(root, filename)
                  for filename in files if filename.endswith("libsvm")]

len(libsvm_file_paths)

In [ ]:
for filename in all_files:
    split = filename.rsplit("/", 1)
    new_dir = split[0] + "_libsvm/"
    if not os.path.exists(new_dir):
        os.makedirs(new_dir)
    os.rename(filename, new_dir + split[1])

# Merge

In [ ]:
import os

with open("merge-files.sh", 'w') as f:
    for root, subdirs, files in os.walk("./"):
        t = [os.path.join(root, filename)
             for filename in files if filename.endswith("libsvm")]
        if t:
            command = "cat %s > %s/data.libsvm" % (' '.join(t), root)
            f.write(command + '\n')

In [ ]:
# Removed individual files

for root, subdirs, files in os.walk("./"):
    for filename in files:
        if filename.endswith("libsvm") and filename != "data.libsvm":
            os.remove(os.path.join(root, filename))

# Shuffle

In [ ]:
import os

all_files = []
for root, subdirs, files in os.walk("./"):
    all_files += [os.path.join(root, filename)
                  for filename in files if filename == "data.libsvm"]

len(all_files)

In [ ]:
from random import randint
from random import shuffle

def shuffle_limited_memory(filename, ntest, nparts):
    assert(ntest < nparts)
    subfiles = [filename + "_part%d" % i for i in range(nparts)]
    handlers = [open(name, 'w') for name in subfiles]
    with open(filename) as f:
        for line in f:
            t = randint(0, nparts - 1)
            handlers[t].write(line)
    for handler in handlers:
        handler.close()

    base = filename.rsplit("/", 1)[0]
    training = open(base + "/training.libsvm", 'w')
    testing = open(base + "/testing.libsvm", 'w')
    shuffle(subfiles)
    for i, name in enumerate(subfiles):
        with open(name) as f:
            lines = f.readlines()        
        os.remove(name)
        shuffle(lines)
        s = ''.join(lines)
        if not s.endswith('\n'):
            s += '\n'
        if i < ntest:
            testing.write(s)
        else:
            training.write(s)
    training.close()
    testing.close()

In [ ]:
for filename in all_files:
    print(filename)
    shuffle_limited_memory(filename, 10, 100)

# Move files

In [ ]:
for filename in all_files:
    old_dir, fname = filename.rsplit("/", 1)
    new_dir = old_dir + "_data"
    os.mkdir(new_dir)
    os.rename(filename, os.path.join(new_dir, fname))

# Upload to S3

In [ ]:
import os

with open("upload-s3.sh", 'w') as f:
    for root, subdirs, files in os.walk("./"):
        if root.endswith("_libsvm"):
            dirname = root[2:]
            f.write("aws s3 cp {} s3://tmsn-data/bathymetry/{}/ --recursive\n".format(root, dirname))